In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [2]:
print(torch.__version__)

2.0.0+cu117


In [9]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("digit82/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("digit82/kobart-summarization")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [10]:
text = """
서치GPT 프로젝트는 하이퍼클로바를 네이버 검색에 특화시킨 검색 대규모 언어모델(서치 LLM) '오션'을 백본으로 활용한다.
이를 바탕으로 네이버가 20년간 축적한 사용자의 검색 흐름 데이터를 모델링하여 사용자가 검색 목적을 달성할 수 있게 최적의 경로를 안내,
이를 통해 검색의도와 결과를 더 잘 이해하고 신뢰성이 강조된 답변을 생성할 수 있게 하는 기술이다. 그는 "정보의 신뢰성, 네이버 서비스와의 연결성,
효과적인 정보 제공을 위한 멀티모달 세 가지를 중점으로 프로젝트를 진행 중"이라고 말했다.
이어 "최신성과 정보 간 교차·반복 검증 및 사용자 피드백을 통해 정확성과 신뢰성을 갖춘 정보를 생성하고,
쇼핑·페이·지도 등과 같은 네이버 서비스와의 유기적인 연동을 통해 사용자의 검색 의도에 최적화된 정보를 이미지·동영상·음성 등과 함께 활용해 직관적인 형태로 제공하고자 한다"며
"서치GPT 프로젝트를 통해 사용자들에게 한층 더 차별화된 검색 경험을 제공할 수 있을 것으로 기대한다"고 말했다.
한편, 2008년 네이버의 사내기술행사로 시작해 2010년 외부 개발자에게 문호를 개방한 데뷰는 매년 IT 분야의 최신 기술과 트렌드를 공유하며
국내·외 다양한 분야의 개발자들이 주목하는 대표적인 개발자 컨퍼런스로 자리매김했다. 올해 15회차를 맞이한 데뷰2023은 27일부터
이틀간 삼성동 코엑스에서 3,500여 명이 참가자들과 함께 진행되며, AI/ML, 클라우드, 웹, 검색, 모바일, NLP, 데이터, 추천, 인프라 등 총 46개 세션으로 구성됐다.
"""

text = text.replace("\n", " ")

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

"서치GPT 프로젝트는 하이퍼클로바를 네이버 검색에 특화시킨 검색 대규모 언어모델(서치 LLM) '오션'을 백본으로 활용해 사용자가 검색 목적을 달성할 수 있게 최적의 경로를 안내, 이를 통해 검색의도와 결과를 더 잘 이해하고 신뢰성이 강조된 답변을 생성할 수 있게 하는 기술이다."

In [14]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

# ====================================================

# 데이터 전처리

- AI-Hub 논문자료 요약 데이터 사용
- https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=90

In [3]:
# 필요한 라이브러리 import
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
from transformers import AdamW
from tqdm import tqdm
import pandas as pd
import numpy as np

In [137]:
def json_folder_to_dataframe(folder_path):
    import os
    import json
    import pandas as pd
    import torch
    from tqdm import tqdm
    
    """
    폴더 안에 있는 JSON 파일을 읽어서 하나의 데이터프레임으로 변환하는 함수
    
    :param folder_path: JSON 파일들이 있는 폴더 경로
    :return: JSON 파일의 데이터를 저장한 데이터프레임
    """
    
    # JSON 파일들의 경로를 리스트로 저장
    file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith('.json')]
    
    # 데이터를 담을 리스트 생성
    original_text = []
    summary_text = []

    # JSON 파일에서 데이터 추출하여 데이터프레임으로 저장
    for file_path in tqdm(file_paths):
        with open(file_path, 'r', encoding = 'utf-8') as f:
            json_data = json.load(f)

        # 데이터 추출
        '''
        데이터에 있는 리스트들을 반복문으로 가져오기
        가져온 리스트 안에 있는 딕셔너리 중 summary_entire의 value 가져오기
        '''
        for i in json_data['data']:
            entire = i['summary_section']

            '''
            가져온 value는 리스트 형태
            최종적으로 그 리스트에 있는 original_text와 summary_text 추출
            '''
            
            for j in entire:
                temp_original = j["orginal_text"]
                temp_summary = j["summary_text"]

                original_text.append(temp_original)
                summary_text.append(temp_summary)

    # 데이터프레임으로 저장
    df = pd.DataFrame([original_text, summary_text], index = ["original", "summary"]).T
    
    # 파일로 저장
    df.to_csv("./data/total.csv", index = False)
    
    
    return df

In [138]:
folder_path = "./data/Total"
df = json_folder_to_dataframe(folder_path)
df

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.20it/s]


,original,summary
0,한국교총은 ‘교사가 된 것을 후회한다.’는 교사 비율이 우리의 경우 20.1%로 조...,한국교총은 교사의 사기를 진작할 특단의 대책을 마련할 것을 촉구하였다. 한국교총은 ...
1,"2015년 8월 18일, 새로운 수학적 발견에 관한 기사가 각 신문마다 게재되었다....",2015년 8월 18일 평면 테셀레이션(Tessellation)이 가능한 15번째의...
2,"정책과정 상 정책변동 이론으로는 전통적으로 합리모형, 만족모형, 점증모형, 쓰레기통...","정책과정상 정책변동 이론으로는 전통적으로 합리모형, 만족모형, 점증모형, 쓰레기통모..."
3,현대 사회에서는 복잡한 문제를 해결하기 위하여 논리적이고 절차적인 사고를 구현하며 ...,미래를 살아갈 학생들에게 나날이 중요성이 더해지는 핵심 역량인 컴퓨팅사고력을 위하여...
4,우리나라 교육정책에서 창의· 인성교육이 교육과정으로 구체화된 것은 2009년 미래형...,오늘날 학교현장은 위주의 교육 현실은 많은 문제점을 낳고 있다. 교육자들은 교육 개...
...,...,...
162336,이 연구는 전국적인 데이터를 활용하여 저소득층 학생의 교육 성취에 대한 사회적 자본...,이 연구는 전국적인 데이터를 활용하여 저소득층 학생의 교육 성취에 대한 사회적 자본...
162337,사건에 대한 정서적인 정보는 이와 같은 기억의 처리과정에 영향을 미칠 수 있는 중요...,"정서가 기억처리과정에 도움을 준다고 주장하는 이들은 정서자극이 중립자극보다 친숙성,..."
162338,최근에 이뤄진 몇 차례의 수학과 교육과정 개정 노력의 결과를 살펴볼 수 있는 방법 ...,"TIMSS와 PISA 결과 분석 연구에 따르면, 우리나라 학생들의 수학 성취는 인지..."
162339,2015년 통계청의 아동 청소년 대상 학업 관련 스트레스 정도 조사에 따르면 매우 ...,학업 상황에서의 스트레스로 인한 문제를 극복하는 대안으로 학업탄력성이 제시되고 있다...


In [4]:
df = pd.read_csv("./data/total.csv")
df

,original,summary
0,한국교총은 ‘교사가 된 것을 후회한다.’는 교사 비율이 우리의 경우 20.1%로 조...,한국교총은 교사의 사기를 진작할 특단의 대책을 마련할 것을 촉구하였다. 한국교총은 ...
1,"2015년 8월 18일, 새로운 수학적 발견에 관한 기사가 각 신문마다 게재되었다....",2015년 8월 18일 평면 테셀레이션(Tessellation)이 가능한 15번째의...
2,"정책과정 상 정책변동 이론으로는 전통적으로 합리모형, 만족모형, 점증모형, 쓰레기통...","정책과정상 정책변동 이론으로는 전통적으로 합리모형, 만족모형, 점증모형, 쓰레기통모..."
3,현대 사회에서는 복잡한 문제를 해결하기 위하여 논리적이고 절차적인 사고를 구현하며 ...,미래를 살아갈 학생들에게 나날이 중요성이 더해지는 핵심 역량인 컴퓨팅사고력을 위하여...
4,우리나라 교육정책에서 창의· 인성교육이 교육과정으로 구체화된 것은 2009년 미래형...,오늘날 학교현장은 위주의 교육 현실은 많은 문제점을 낳고 있다. 교육자들은 교육 개...
...,...,...
162336,이 연구는 전국적인 데이터를 활용하여 저소득층 학생의 교육 성취에 대한 사회적 자본...,이 연구는 전국적인 데이터를 활용하여 저소득층 학생의 교육 성취에 대한 사회적 자본...
162337,사건에 대한 정서적인 정보는 이와 같은 기억의 처리과정에 영향을 미칠 수 있는 중요...,"정서가 기억처리과정에 도움을 준다고 주장하는 이들은 정서자극이 중립자극보다 친숙성,..."
162338,최근에 이뤄진 몇 차례의 수학과 교육과정 개정 노력의 결과를 살펴볼 수 있는 방법 ...,"TIMSS와 PISA 결과 분석 연구에 따르면, 우리나라 학생들의 수학 성취는 인지..."
162339,2015년 통계청의 아동 청소년 대상 학업 관련 스트레스 정도 조사에 따르면 매우 ...,학업 상황에서의 스트레스로 인한 문제를 극복하는 대안으로 학업탄력성이 제시되고 있다...


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162341 entries, 0 to 162340
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   original  162341 non-null  object
 1   summary   162335 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [141]:
df.isna().sum()

original    0
summary     6
dtype: int64

In [142]:
df["original"].value_counts()

의료 분야에 인공지능(Artificial Intelligence) 기술에 대한 적용이 활발하게 이루어지고 있다. 의사에 의한 진료가 아직까지 더 정확할 수 있으나 누적된 진료 기록이 존재한다면 학습을 통해 일정한 패턴을 갖는 질병을 예측할 수 있을 것이다. 심장병의 경우 가슴 통증, 혈압, 심박수, 혈관수 등의 진료 패턴이 존재하여 실제 심장병 환자의 진료 정보를 이용한다면 심장병에 대한 예측이 가능할 것이다. 또한 고비용의 혈관조영술도 비심장병 의심 환자들이 검사를 받을 필요가 없을 것이다. 본 연구에서는 이러한 일정한 패턴을 가진 심장병 환자의 진료 데이터를 기반으로 인공지능의 머신러닝(Machine Learning) 기술인 딥러닝(Deep Learning)을 통해 심장병 예측을 연구한다. 1장에서는 본 연구의 심장병 예측의 필요성을 설명한다. 2장에서는 관련 연구로서 딥러닝 기술을 파악하고, 심장병 관련 기존 연구들을 분석한다. 3장에서는 심장병 예측을 위한 심장병 환자의 데이터 정보와 딥러닝 기반의 다층 퍼셉트론을 제시하고 딥러닝의 개발 단계를 제시한다. 4장에서는 구글의 텐서플로를 이용하여 심장병 예측을 실험하며, 기존의 연구들과 정확도를 비교 분석한다. 5장에서 결론 및 향후 연구 과제를 제시한다.                                                                                                                                              4
철학이 걸어왔던 길을 살펴보면, 어느 한 때도 위기의 순간이 아니었던 적은 없었다. 사실 철학은 위기에서 출발하는 것이 아닌가? 왜냐하면 철학함이란 바로 질문하는 것에서 출발하기 때문이다. 질문한다는 것은 기존 질서에 대해 의문을 제기하는 것이고, 새로운 도전이라 할 수 있으며, 또한 새롭게 방향을 모색하기 위한 반성의 작용이라고 할 수 있다. 만약 기존의 체제나 삶의 방식에 순응하고 있거나, 답습하려는

In [143]:
df.drop_duplicates()

,original,summary
0,한국교총은 ‘교사가 된 것을 후회한다.’는 교사 비율이 우리의 경우 20.1%로 조...,한국교총은 교사의 사기를 진작할 특단의 대책을 마련할 것을 촉구하였다. 한국교총은 ...
1,"2015년 8월 18일, 새로운 수학적 발견에 관한 기사가 각 신문마다 게재되었다....",2015년 8월 18일 평면 테셀레이션(Tessellation)이 가능한 15번째의...
2,"정책과정 상 정책변동 이론으로는 전통적으로 합리모형, 만족모형, 점증모형, 쓰레기통...","정책과정상 정책변동 이론으로는 전통적으로 합리모형, 만족모형, 점증모형, 쓰레기통모..."
3,현대 사회에서는 복잡한 문제를 해결하기 위하여 논리적이고 절차적인 사고를 구현하며 ...,미래를 살아갈 학생들에게 나날이 중요성이 더해지는 핵심 역량인 컴퓨팅사고력을 위하여...
4,우리나라 교육정책에서 창의· 인성교육이 교육과정으로 구체화된 것은 2009년 미래형...,오늘날 학교현장은 위주의 교육 현실은 많은 문제점을 낳고 있다. 교육자들은 교육 개...
...,...,...
92077,"일제강점기 재만 조선인문학을 논의하면서 흔히 제기되는 문제는 친일문학, 어용문학, ...",호미바바의 혼종성 이론은 복잡한 양상을 띠는 일제강점기 재만 조선인문학 논의에서 식...
92078,수능은 대학 입학을 위한 고부담 시험이어서 대국민적 관심사이다. 그러다 보니 학생들...,사고력 중심의 평가였던 초창기 수능 언어 영역의 영향으로 각 학교는 수능 성적을 향...
92079,본 연구에서는 이러한 관점에 입각하되 구체적으로 ‘국어과 정의적 영역의 평가’와 관...,본 연구는 경기도교육과정 평가 혁신 과제인 ‘국어과 정의적 영역의 평가’ 정책에 초...
92080,"의미의 본질을 단어, 문장, 텍스트로 확장하여 함께 살핀다는 것은 일상 언어의 측면...","본 연구는 국어교육에서의 의미 교육의 양상을 검토하고, 의미의 역동성 개념에서 의..."


In [5]:
ndf = df.dropna().drop_duplicates(keep = "last").reset_index(drop = True)
ndf

,original,summary
0,이상에서 필자는 몰입이론을 진로상담에 적용하여 새로운 진로상담 모형을 모색해보았다....,이상에서 필자는 몰입이론을 진로상담에 적용하여 새로운 진로상담 모형을 모색해보았고 ...
1,청소년의 정신건강과 복지를 위한 각종 기관의 설립 및 서비스 사업이 활발하게 이루어...,청소년의 정신건강과 복지를 위한 각종 기관의 설립 및 서비스 사업이 활발하게 이루어...
2,이상과 같은 결과들을 토대로 살펴본 본 연구의 의의 및 시사점은 다음과 같다. 우선...,"자살행동의 저연령화에 충격을 받고 있는 시기에, 본 연구는 초등학교 아동들을 대상으..."
3,본 연구는 자기애적 성격성향과 비행성향에 대하여 알아보고자 하였다. 본 연구에서 다...,자기애적 성격성향과 비행성향에 대하여 알아보고자 하였고 이런 구성개념은 자기애와 청...
4,대부분의 음란물은 성폭력(sexual violence)을 담고 있다. 음란물에서 나...,음란물에서 나타나는 전형적인 성폭력 상황은 여성이 처음에는 남성의 공격에 강하게 저...
...,...,...
92061,이 연구는 전국적인 데이터를 활용하여 저소득층 학생의 교육 성취에 대한 사회적 자본...,이 연구는 전국적인 데이터를 활용하여 저소득층 학생의 교육 성취에 대한 사회적 자본...
92062,사건에 대한 정서적인 정보는 이와 같은 기억의 처리과정에 영향을 미칠 수 있는 중요...,"정서가 기억처리과정에 도움을 준다고 주장하는 이들은 정서자극이 중립자극보다 친숙성,..."
92063,최근에 이뤄진 몇 차례의 수학과 교육과정 개정 노력의 결과를 살펴볼 수 있는 방법 ...,"TIMSS와 PISA 결과 분석 연구에 따르면, 우리나라 학생들의 수학 성취는 인지..."
92064,2015년 통계청의 아동 청소년 대상 학업 관련 스트레스 정도 조사에 따르면 매우 ...,학업 상황에서의 스트레스로 인한 문제를 극복하는 대안으로 학업탄력성이 제시되고 있다...


In [145]:
ndf.isna().sum()

original    0
summary     0
dtype: int64

In [6]:
# 데이터셋 분리 (8 : 2)
total_records = ndf.shape[0]
test_records = int(total_records * .2)

In [7]:
train_data = ndf[: total_records - test_records]
test_data = ndf[total_records - test_records : total_records]

In [8]:
train_data.to_csv("./data/train.csv", index = False)
test_data.to_csv("./data/test.csv", index = False)

In [129]:
ndf["original"].str.len().sort_values().tail(20)

76542    1011
84625    1015
9102     1016
27424    1017
81552    1019
58970    1021
49390    1027
49243    1029
49404    1037
49664    1048
80838    1050
36861    1052
45806    1056
38305    1056
77053    1078
69772    1088
44525    1132
49672    1145
1433     1254
79522    1447
Name: original, dtype: int64